### Bert loading

In [46]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt


In [38]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.load_state_dict(torch.load("/kaggle/working/bert_sst2.pth"))

model.eval()

tokenizer = BertTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() 

print("Model loaded successfully!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!


Integrated Gradients is an attribution method that aims to explain the relationship between a model's predictions and its inputs. It assigns an "importance score" to each feature of the input (in our case, each token) regarding a specific prediction.

The main idea is

How much did a particular token contribute to the final prediction, compared to a neutral baseline?

Baseline==A neutral or "information-less" input , space toeksn , end of sentence ect

Gradients==The gradient of the model's output with respect to the input

Integration==we dotn pick a random starting point to inspect the gradient we sum it uo from each step 

In [22]:
# class0==neg
#class 1==pos

## what is shap

In [32]:
!pip install shap


In [35]:
model = model.to(device)

In [52]:
def predict(texts):
    #how finally fix the list /str error that made me cry
    texts = [" ".join(t) if isinstance(t, list) else t for t in texts]

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
    
    return probs.cpu().numpy()


masker = shap.maskers.Text(tokenizer)
explainer = shap.Explainer(predict, masker)
shap_values = explainer(["This movie is good"])
shap.plots.text(shap_values[0])


## sharp interpretation

the words in blue are what is adding strong contribution to the positive meaning (this /good)


In [53]:

shap_values = explainer(["This movie is bad"])
shap.plots.text(shap_values[0])
